##SUPERMARKET SALES ANALYSIS

Importando librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import numpy

Variables Globales

In [2]:
global df_ventas
global df_ventas_limpio

*Importando la Base de Datos .csv*

In [3]:
def obtencion_datos():
    global df_ventas
    df_ventas = pd.read_csv('reporte_ventas.csv', sep = ',')
obtencion_datos()

Normalización y limpieza de los datos

In [4]:
#Formateamos el tipo de dato de las fechas a uno adecuado
df_ventas['fecha_pedido'] = pd.to_datetime(df_ventas['fecha_pedido'], format='%Y-%m-%d')
df_ventas['fecha_envio'] = pd.to_datetime(df_ventas['fecha_envio'], format='%Y-%m-%d')

#Eliminar repetidos
df_ventas_limpio = df_ventas.drop_duplicates()

#Eliminamos registros de prueba
def eliminar_registros_prueba(df):
  return df.drop(df[df['nombre_cliente'] == 'prueba'].index)
df_ventas_limpio = eliminar_registros_prueba(df_ventas_limpio)

#Reemplazamos los nombres de clientes NaN por el valor "missing"
df_ventas_limpio['nombre_cliente'].fillna('missing', inplace=True)

In [5]:
# Filtrar los registros con total_ganancia > 0
df_ventas_limpio = df_ventas.drop(df_ventas[df_ventas["total_ganancia"]  <= 0].index)

#reiniciamos el index
df_ventas_limpio.reset_index(inplace=True)
df_ventas_limpio.drop(columns=['index'], inplace=True)

In [ ]:
df_ventas_limpio

In [ ]:
#Code here


##PREPARACIÓN DE LOS DATOS

2. ¿Cuáles son los 7 productos que más ganancias mostraron durante todo el periodo?

In [7]:
#Función pregunta_2()
def pregunta_2():
    # Definimos las variables globales
    global df_ventas
    global df_ventas_limpio

In [8]:

df_top_ganancias = df_ventas_limpio[['tipo_producto', 'total_ganancia']].copy()

In [ ]:

df_top_ganancias = df_top_ganancias.groupby('tipo_producto').agg({'total_ganancia': 'sum'}).reset_index()

df_top_ganancias = df_top_ganancias.sort_values(by='total_ganancia', ascending=False)

df_top_ganancias = df_top_ganancias.head(7)

# Imprime el resultado
print(df_top_ganancias.to_string(index=False))


##VISUALIZACIÓN

In [ ]:
# Configuración de la figura y el eje
fig, ax = plt.subplots(figsize=(10, 4))

ejeX = df_top_ganancias['tipo_producto']
ejeY = df_top_ganancias['total_ganancia']

colores = ['#062075','#40036e','#0077b6','#0C8040','#9c2e09', '#5c0404','#565e66']

barras = ax.barh(ejeX, ejeY, color=colores, align='center')

for barra in barras:
  width = barra.get_width()
  ax.text(width, barra.get_y() + barra.get_height() / 2, f'USD ${width:.2f}', ha='right', va='center', color='white', weight='bold')

ax.set_title('Top 7 productos con mayor ganancia (2019-2022)')
ax.set_xlabel('Total de Ganancias')
ax.set_ylabel('Tipo de Producto')

# Mostrar el gráfico
plt.show()


1. ¿Cuáles son las ventas totales por año? ¿Y qué año tuvo mejor desempeño? (Pablo)

3. ¿Si separamos las ventas totales por año y por regiones ¿Qué insight podemos obtener de nuestras ventas? (Jhojan)

4. ¿Cuál es el método de envío más utilizado por los clientes del Supermercado? ¿Es proporcional para B2B y B2C?

5. ¿Cuáles son las ventas totales por trimestre en el estado de Sao Paulo? (Everardo)

In [ ]:
#Code here


6. ¿Existe otro insight que puedas proporcionar?